In [2]:
import torch
import numpy as np
import torch.nn as nn

In [47]:
class modules(nn.Module):
    def __init__(self, layers):
        
        super(modules, self).__init__()
        
        self.layer1 = nn.Sequential(
            *layers
                                    )
        
        
    def forward(self, x):
        
        out = self.layer1(x)
        
        return out

In [78]:
class Net(nn.Module):
    def __init__(self, modules):
        
        super(Net, self).__init__()
        self.c=[]
        for i in modules:
            for j in i.children():
                self.c.append(j)
#                 j.register_forward_hook(self.j(x))
                
#         self.name , module = modules
        self.layer1 = nn.Sequential(*self.c)
        
        
    def forward(self,x):
        f=[]
        super(Net, self).__init__()

        for i in self.c:
            out1 = i(x)
            f.append(out1)

#         out1,out2,out3 = self.layer1(x)
        
        out=torch.cat(f)
        
        return out
    

In [75]:
class mk_modules(nn.Module):
    def __init__(self, layers):
        super(mk_modules, self).__init__()
        
        self.name, layer = layers

        setattr(self, self.name, nn.Sequential(*layer))
        
        print(layer)
    def forward(self, x ):
        
        method = getattr(self, self.name)
        out = method(x)
        
        return out

In [76]:
a.forward

<bound method Net.forward of Net(
  (layer1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
    (2): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)>

In [81]:
tmp = []
inchannels = 3
# for i in range(2):
layer = ('_1x1',[nn.Conv2d(in_channels=inchannels,out_channels=24,kernel_size=1)])
# tmptmp=mk_modules(layer)
tmp.append(mk_modules(layer))

layer = ('_5x5',[nn.Conv2d(in_channels=inchannels,out_channels=16,kernel_size=1),
               nn.Conv2d(in_channels=16,out_channels=24,kernel_size=5,padding=2)])
tmp.append(mk_modules(layer))

layer = ('_3x3',[nn.Conv2d(in_channels=inchannels,out_channels=16,kernel_size=1),
               nn.Conv2d(in_channels=16,out_channels=24,kernel_size=3,padding=1),
               nn.Conv2d(in_channels=24,out_channels=24,kernel_size=3,padding=1)])
tmp.append(mk_modules(layer))



a=Net(tmp)
x=torch.rand([10,3,98,98])
a(x).shape

[Conv2d(3, 24, kernel_size=(1, 1), stride=(1, 1))]
[Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1)), Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))]
[Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1)), Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))]


torch.Size([30, 24, 98, 98])

In [45]:
model = nn.Sequential()
model.add_module('conv0', tmp[0])
model.add_module('norm0', tmp[1])
model.add_module('norm1', tmp[2])

In [ ]:
module.register_forward_hook(self,hook_fn)

In [46]:
model

Sequential(
  (conv0): mk_modules(
    (_1x1): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (norm0): mk_modules(
    (_5x5): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
  )
  (norm1): mk_modules(
    (_3x3): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)

In [29]:
model=nn.Sequential(nn.Linear(4,5),nn.Sequential(nn.Linear(4,5),
    
    nn.Sequential(nn.Linear(3,2),
nn.Conv2d(3,30,2),
nn.Linear(2,4))))

In [38]:
module.register_forward_hook(self,hook_fn)

for j, i in model.named_modules():
#     print(i)
#     break
    if isinstance(i, nn.Conv2d):
        print(i.weight.shape)
        print(i.bias.shape)
        break

torch.Size([30, 3, 2, 2])
torch.Size([30])


In [196]:
1 . 리스트에 담에서 한번에 추가한다?
2 . 한개씩 순서에 맞게 정리?

'_1x1'

In [203]:
a

Net(
  (layer1): Sequential(
    (0): mk_modules(
      (_1x1): Sequential(
        (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): mk_modules(
      (_5x5): Sequential(
        (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      )
    )
    (2): mk_modules(
      (_3x3): Sequential(
        (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
)

In [190]:
a.layer1._modules['0']

mk_modules(
  (_1x1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [141]:
a.forward

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__seta

In [102]:
name = 'd'
d='f'
s=globals()['{}'.format(name)]
s

'f'

In [31]:
b

Net(
  (layer1): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1))
      (2): Linear(in_features=6, out_features=3, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1))
      (2): Linear(in_features=6, out_features=3, bias=True)
    )
  )
)

In [10]:
for i in layers:
    if isinstance(i, nn.Linear):
        print(i.weight.shape)

torch.Size([3, 2])


In [14]:
if isinstance(layers, nn.Linear):
    print(layers.weight.shape)